In [1]:
from PIL import Image
import os
import pdb
import sys
import numpy as np
import pandas as pd
import glob
import librosa
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


sys.path.append('/Users/annayang/Documents/QueenBee/Bee_NotBee_classification')

from utils import raw_feature_fromSample, get_samples_id_perSet, get_GT_labels_fromFiles, labels2binary, load_audioFiles_saves_segments, write_Statelabels_from_samplesFolder, get_list_samples_names, split_samples_ramdom

In [2]:
print(librosa.__version__)

0.9.2


In [3]:
new_data = pd.read_csv('/Users/annayang/Documents/QueenBee/data_combined_training_2.csv').reset_index(drop=True).drop('Unnamed: 0', axis = 1)
new_data

,device,date,hive temp,hive humidity,hive pressure,weather temp,weather humidity,weather pressure,wind speed,gust speed,weatherID,cloud coverage,rain,lat,long,file name,queen presence,queen acceptance,frames,target
0,1,2022-06-24 10:03:35,42.76,38.83,1005.69,25.03,43,1012,2.57,NaN,800,0,0,37.29,-121.95,2022-06-24--10-03-35_1.raw,1,2,8,5
1,1,2022-06-24 11:03:30,48.42,20.98,1005.54,27.45,38,1011,3.60,NaN,800,0,0,37.29,-121.95,2022-06-24--11-03-30_1.raw,1,2,8,5
2,1,2022-06-24 12:02:42,47.41,18.39,1005.24,30.37,34,1011,3.09,NaN,800,0,0,37.29,-121.95,2022-06-24--12-02-42_1.raw,1,2,8,5
3,1,2022-06-24 13:01:08,44.55,20.51,1004.74,32.40,35,1010,4.12,NaN,800,0,0,37.29,-121.95,2022-06-24--13-01-08_1.raw,1,2,8,5
4,1,2022-06-24 14:00:07,38.81,25.14,1004.50,33.06,35,1010,4.63,NaN,800,0,0,37.29,-121.95,2022-06-24--14-00-07_1.raw,1,2,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,2,2022-07-15 11:18:39,50.84,11.99,1010.21,23.58,55,1015,2.57,NaN,800,0,0,37.29,-121.95,2022-07-15--11-18-39_2.raw,1,2,10,5
971,2,2022-07-15 12:25:14,49.58,11.60,1009.81,25.60,51,1015,4.12,NaN,800,0,0,37.29,-121.95,2022-07-15--12-25-14_2.raw,1,2,10,5
972,2,2022-07-15 13:25:58,45.83,15.36,1009.80,26.49,49,1015,5.14,NaN,800,0,0,37.29,-121.95,2022-07-15--13-25-58_2.raw,1,2,10,5
973,2,2022-07-15 14:24:58,35.82,23.48,1009.26,27.33,46,1014,4.63,NaN,800,0,0,37.29,-121.95,2022-07-15--14-24-58_2.raw,1,2,10,5


# converting data from raw to wav

In [51]:
import wave

for i in new_data['file name']:
    filename = i[:-4]
    with open("/Users/annayang/Documents/QueenBee/newData_raw/"+i, "rb") as inp_f:
        data = inp_f.read()
        with wave.open("/Users/annayang/Documents/QueenBee/newData/"+filename+".wav", "wb") as out_f:
            out_f.setnchannels(1)
            out_f.setsampwidth(2)
            out_f.setframerate(16000)
            out_f.writeframesraw(data)

In [4]:
with wave.open('../newData/2022-06-15--13-28-45_2.wav', 'rb') as in_f:
    print(repr(in_f.getparams()))

NameError: name 'wave' is not defined

# creating raw state labels

In [4]:
raw_state_labels = new_data.loc[:, 'file name': 'queen presence'].copy()
raw_state_labels

,file name,queen presence
0,2022-06-24--10-03-35_1.raw,1
1,2022-06-24--11-03-30_1.raw,1
2,2022-06-24--12-02-42_1.raw,1
3,2022-06-24--13-01-08_1.raw,1
4,2022-06-24--14-00-07_1.raw,1
...,...,...
970,2022-07-15--11-18-39_2.raw,1
971,2022-07-15--12-25-14_2.raw,1
972,2022-07-15--13-25-58_2.raw,1
973,2022-07-15--14-24-58_2.raw,1


In [5]:
raw_state_labels['file name'] = raw_state_labels['file name'].str[:-4]
raw_state_labels

,file name,queen presence
0,2022-06-24--10-03-35_1,1
1,2022-06-24--11-03-30_1,1
2,2022-06-24--12-02-42_1,1
3,2022-06-24--13-01-08_1,1
4,2022-06-24--14-00-07_1,1
...,...,...
970,2022-07-15--11-18-39_2,1
971,2022-07-15--12-25-14_2,1
972,2022-07-15--13-25-58_2,1
973,2022-07-15--14-24-58_2,1


In [6]:
raw_state_labels.to_csv('../newDataset/raw_state_labels.csv')

# generating segments and state labels

In [7]:
block_size=60
path_audioFiles="/Users/annayang/Documents/QueenBee/newData"+os.sep
path_save_audio_labels='/Users/annayang/Documents/QueenBee/newDataset'+os.sep  # path where to save audio segments and labels files.
path_raw_state_labels = '/Users/annayang/Documents/QueenBee/newDataset/raw_state_labels.csv'
path_state_labels = '/Users/annayang/Documents/QueenBee/newDataset/state_labels.csv'

In [8]:
raw_state_labels = pd.read_csv(path_raw_state_labels)
audiofilenames_list = [os.path.basename(x) for x in glob.glob(path_audioFiles+'*.wav')]
load_audioFiles_saves_segments(audiofilenames_list, path_audioFiles,
    path_save_audio_labels, block_size , [0], '-', read_beeNotBee_annotations='no', save_audioSegments='yes')

write_Statelabels_from_samplesFolder(path_save_audio_labels, path_save_audio_labels, raw_state_labels)

sample_ids=get_list_samples_names(path_audioFiles) # get sample ids from non-segmented folder.
state_labels = pd.read_csv('/Users/annayang/Documents/QueenBee/newDataset/state_labels.csv')

[2022-09-28 12:46:52 RAM65.9% 0.2GB] Number of audiofiles in folder: 975


[2022-09-28 12:46:52 RAM65.9% 0.2GB] Processing 2022-06-26--00-55-43_1.wav          :::file number:  1 --------->of 975
-----------------Reading segment 0
2022-06-26--00-55-43_1__segment0
-----------------Saved wav file for segment 0
-----------------Reading segment 1
2022-06-26--00-55-43_1__segment1
-----------------Saved wav file for segment 1
-----------------Reading segment 2
2022-06-26--00-55-43_1__segment2
-----------------Saved wav file for segment 2
-----------------Reading segment 3
2022-06-26--00-55-43_1__segment3
-----------------Saved wav file for segment 3
-----------------Reading segment 4
2022-06-26--00-55-43_1__segment4
-----------------Saved wav file for segment 4
-----------------Reading segment 5
2022-06-26--00-55-43_1__segment5
-----------------Uniformizing block length of segment5
-----------------Saved wav file for segment 5


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


----------------- no more segments for this file--------------------------------------




[2022-09-28 12:47:10 RAM66.8% 0.26GB] Processing 2022-07-08--19-09-34_2.wav          :::file number:  2 --------->of 975
-----------------Reading segment 0
2022-07-08--19-09-34_2__segment0
-----------------Saved wav file for segment 0
-----------------Reading segment 1
2022-07-08--19-09-34_2__segment1
-----------------Saved wav file for segment 1
-----------------Reading segment 2
2022-07-08--19-09-34_2__segment2
-----------------Saved wav file for segment 2
-----------------Reading segment 3
2022-07-08--19-09-34_2__segment3
-----------------Saved wav file for segment 3
-----------------Reading segment 4
2022-07-08--19-09-34_2__segment4
-----------------Saved wav file for segment 4
-----------------Reading segment 5
2022-07-08--19-09-34_2__segment5
-----------------Uniformizing block length of segment5
-----------------Saved wav file for segment 5
----------------- no more segments for this file-

-----------------Reading segment 3
2022-07-04--01-58-01_1__segment3
-----------------Saved wav file for segment 3
-----------------Reading segment 4
2022-07-04--01-58-01_1__segment4
-----------------Saved wav file for segment 4
-----------------Reading segment 5
2022-07-04--01-58-01_1__segment5
-----------------Uniformizing block length of segment5
-----------------Saved wav file for segment 5
----------------- no more segments for this file--------------------------------------




[2022-09-28 12:49:53 RAM67.2% 0.26GB] Processing 2022-07-13--13-45-31_1.wav          :::file number:  11 --------->of 975
-----------------Reading segment 0
2022-07-13--13-45-31_1__segment0
-----------------Uniformizing block length of segment0
-----------------Saved wav file for segment 0
----------------- no more segments for this file--------------------------------------




[2022-09-28 12:49:55 RAM67.2% 0.26GB] Processing 2022-07-06--22-10-00_1.wav          :::file number:  12 --------->of 975
--------

-----------------Saved wav file for segment 5
----------------- no more segments for this file--------------------------------------




[2022-09-28 13:42:13 RAM57.0% 0.26GB] Processing 2022-07-08--19-41-57_1.wav          :::file number:  20 --------->of 975
-----------------Reading segment 0
2022-07-08--19-41-57_1__segment0
-----------------Saved wav file for segment 0
-----------------Reading segment 1
2022-07-08--19-41-57_1__segment1
-----------------Saved wav file for segment 1
-----------------Reading segment 2
2022-07-08--19-41-57_1__segment2
-----------------Saved wav file for segment 2
-----------------Reading segment 3
2022-07-08--19-41-57_1__segment3
-----------------Saved wav file for segment 3
-----------------Reading segment 4
2022-07-08--19-41-57_1__segment4
-----------------Saved wav file for segment 4
-----------------Reading segment 5
2022-07-08--19-41-57_1__segment5
-----------------Uniformizing block length of segment5
-----------------Saved wav file for segment 5
--

...

2022-06-25--12-41-45_1
2022-07-03--15-07-28_1
2022-06-29--15-12-27_2
2022-07-06--05-40-59_2
2022-07-02--04-01-13_1
2022-07-10--18-36-58_2
2022-06-15--15-26-47_2
2022-06-29--10-17-35_2
2022-07-14--17-22-39_1
2022-07-03--21-08-19_2
2022-07-12--23-53-54_1
2022-07-04--06-54-06_1
2022-06-29--16-11-32_2
2022-06-28--01-20-12_1
2022-07-14--14-01-51_2
2022-07-06--06-40-12_2
2022-07-01--13-40-23_1
2022-07-10--23-32-49_2
2022-06-24--18-55-31_1
2022-07-06--18-30-44_2
2022-07-05--04-39-52_1
2022-07-14--17-01-03_2
2022-06-29--05-20-23_2
2022-07-04--05-54-54_1
2022-07-04--09-52-21_1
2022-06-29--12-16-53_1
2022-06-16--19-02-57_2
2022-07-05--11-34-47_1
2022-07-11--09-27-27_2
2022-06-26--11-47-55_2
2022-06-25--09-44-22_1
2022-06-16--07-13-31_2
2022-06-26--07-50-24_1
2022-07-10--04-41-21_1
2022-07-05--00-43-17_1
2022-06-26--20-40-03_2
2022-06-16--08-12-31_2
2022-07-03--12-11-40_2
2022-07-11--10-20-30_1
2022-06-16--08-12-31_2
2022-06-26--20-40-03_2
2022-07-11--10-20-30_1
2022-07-03--12-11-40_2
2022-07-10-

2022-06-28--23-25-29_2
2022-07-10--12-38-24_1
2022-07-10--12-42-27_2
2022-07-08--04-12-14_2
2022-07-01--16-37-23_1
2022-07-10--01-43-36_1
2022-07-12--07-04-56_1
2022-07-07--22-53-00_1
2022-07-05--02-55-46_2
2022-06-28--10-14-20_2
2022-06-25--19-59-50_1
2022-06-28--11-13-44_1
2022-06-26--00-55-53_2
2022-07-06--03-42-38_2
2022-07-05--13-47-17_2
2022-06-29--02-22-49_1
2022-07-12--10-03-19_1
2022-07-08--20-08-46_2
2022-06-30--03-50-31_2
2022-07-03--01-20-01_2
2022-07-02--12-54-38_1
2022-07-12--12-17-36_2
2022-07-04--17-46-00_1
2022-06-25--06-45-24_1
2022-07-15--03-19-22_1
2022-06-24--21-52-53_2
2022-06-27--04-34-04_1
2022-07-12--10-18-46_2
2022-07-13--01-08-06_2
2022-06-29--09-18-09_2
2022-07-06--08-38-42_2
2022-06-28--04-17-36_2
2022-06-30--00-53-00_2
2022-06-24--11-02-54_2
2022-06-27--04-33-32_2
2022-07-01--00-19-07_1
2022-06-26--05-51-52_1
2022-07-10--08-42-54_2
2022-06-29--16-11-49_1
2022-06-16--11-11-23_2
2022-07-12--12-01-45_1
2022-07-04--08-52-56_1
2022-06-27--12-20-07_1
2022-07-04-

2022-07-10--07-40-40_1
2022-07-03--06-15-59_2
2022-07-07--02-08-26_1
2022-07-07--11-20-34_2
2022-07-03--09-13-44_2
2022-06-28--08-14-40_2
2022-07-14--11-27-51_1
2022-07-13--03-06-31_2
2022-07-08--11-48-51_1
2022-06-30--06-47-59_2
2022-06-26--23-37-32_2
2022-07-12--02-23-55_2
2022-06-16--10-12-01_2
2022-07-08--03-12-29_2
2022-07-12--13-15-19_2
2022-06-15--23-19-48_2
2022-07-03--18-04-54_1
2022-07-10--01-47-45_2
2022-07-02--19-23-00_1
2022-07-04--08-58-57_2
2022-07-09--00-40-46_1
2022-06-26--19-41-01_2
2022-07-03--01-18-05_1
2022-07-06--13-14-32_1
2022-06-28--23-25-19_1
2022-07-06--23-29-23_2
2022-07-08--08-08-29_2
2022-07-15--07-05-32_2
2022-06-25--05-46-18_2
2022-06-27--14-16-54_2
2022-06-15--13-28-45_2
2022-07-04--11-56-52_2
2022-07-10--00-48-36_2
2022-06-26--18-42-02_1
2022-07-12--22-07-06_2
2022-07-01--15-38-20_1
2022-06-16--12-10-39_2
2022-07-05--05-53-35_2
2022-07-12--11-02-27_1
2022-07-02--21-21-14_1
2022-07-07--23-13-22_2
2022-07-09--00-04-21_2
2022-07-03--03-16-26_1
2022-07-05-

2022-06-26--04-52-44_2
2022-06-30--13-43-06_2
2022-07-15--03-08-59_2
2022-07-04--00-58-48_1
2022-07-05--03-54-55_2
2022-07-10--22-33-37_2
2022-07-14--19-59-41_2
2022-07-02--20-24-01_2
2022-06-15--14-27-39_2
2022-06-29--13-15-03_1
2022-07-03--07-15-14_2
2022-07-04--19-47-40_1
2022-07-02--03-01-54_1
2022-07-02--05-00-25_1
2022-07-15--05-17-44_1
2022-06-29--00-24-28_1
2022-06-30--07-47-14_2
2022-07-12--08-04-17_1
2022-07-12--13-01-21_1
2022-07-08--12-08-53_2
2022-07-09--08-34-36_1
2022-06-16--21-01-03_2
2022-07-04--13-55-26_2
2022-07-12--14-14-09_2
2022-07-14--23-22-40_1
2022-07-10--06-39-34_1
2022-07-06--11-36-55_2
2022-06-27--13-17-43_2
2022-06-30--21-21-31_1
2022-07-06--14-34-31_2
2022-06-25--08-44-24_2
2022-06-29--20-56-16_2
2022-06-27--00-36-44_2
2022-06-25--23-56-32_1
2022-07-05--09-36-17_1
2022-07-05--23-24-37_1
2022-06-26--08-49-43_2
2022-07-07--16-17-49_2
2022-07-02--14-52-46_1
2022-07-13--21-38-28_1
2022-07-09--15-34-08_1
2022-07-11--18-20-59_2
2022-06-26--15-44-40_1
2022-07-10-

2022-07-09--07-57-34_2
2022-07-07--17-57-13_1
2022-07-03--04-17-36_2
2022-07-10--07-43-18_2
2022-07-15--01-20-58_1
2022-07-04--22-59-04_2
2022-06-27--20-24-32_1
2022-07-10--10-39-50_1
2022-06-27--18-13-28_2
2022-07-09--03-02-18_2
2022-07-02--08-57-35_1
2022-06-24--12-02-42_1
2022-07-12--09-19-37_2
2022-07-13--21-58-04_2
2022-06-25--22-57-22_1
2022-07-10--19-35-56_2
2022-07-15--12-13-31_1
2022-07-13--13-45-31_1
2022-07-10--14-36-54_1
2022-07-12--16-12-07_2
2022-07-04--02-04-13_2
2022-06-27--00-37-09_1
2022-07-12--20-09-00_2
2022-07-07--04-25-29_2
2022-06-30--08-46-38_2
2022-07-03--06-13-59_1
2022-06-30--08-46-38_2
2022-07-03--06-13-59_1
2022-07-04--02-04-13_2
2022-07-12--16-12-07_2
2022-07-10--14-36-54_1
2022-07-07--04-25-29_2
2022-07-12--20-09-00_2
2022-06-27--00-37-09_1
2022-07-12--09-19-37_2
2022-06-24--12-02-42_1
2022-07-15--12-13-31_1
2022-07-10--19-35-56_2
2022-06-25--22-57-22_1
2022-07-13--21-58-04_2
2022-07-10--10-39-50_1
2022-06-27--20-24-32_1
2022-07-02--08-57-35_1
2022-07-09-

# random split with non-segmented data

In [21]:
for i in range(3):
    split_dict = split_samples_ramdom(0.1,0.2,path_save_audio_labels, path_save_audio_labels+'split_random_'+str(i))

# categorization and image generation
**fix to do the chunking**

In [22]:
sample_ids_test, sample_ids_train, sample_ids_val = get_samples_id_perSet(path_save_audio_labels+'split_random_0.json')

labels2read = 'state_labels'
labels_train = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_train, labels2read)
Y_train= labels2binary('missing queen', labels_train)

labels_val = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_val, labels2read)
Y_val= labels2binary('missing queen', labels_val)

labels_test = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_test, labels2read)
Y_test= labels2binary('missing queen', labels_test)

In [23]:
import os
full_list = os.listdir("/Users/annayang/Documents/QueenBee/newDataset")
len(full_list)

5394

In [24]:
for i, sample in enumerate(sample_ids_test):
    prefix = sample[:-4]
    high_level_features = 0
    raw_feature = 'MFCCs20'
    result = [j for j in full_list if j.startswith(prefix)]
    for filename in result:
        # raw feature extraction:
        x = raw_feature_fromSample(path_save_audio_labels+filename, raw_feature ) # x.shape: (4, 20, 2584)
        x_norm = x

    #     #normalization here:
    #     if not normalization == 'NO':
    #         x_norm = featureMap_normalization_block_level(x, normalizationType = normalization) 
    #     else: x_norm = x

        if high_level_features:
            # high level feature extraction:
            if 'MFCCs' in raw_feature:
                X = compute_statistics_overMFCCs(x_norm, 'yes') # X.shape: (4 , 120)
            else: 
                X = compute_statistics_overSpectogram(x_norm)

            feature_map=X
        else:
            feature_map=x_norm

        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        p = librosa.display.specshow(feature_map, ax=ax, y_axis='log', x_axis='time')

        if Y_test[i] == 0:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/test/0/' + filename[:-4] + '.png')
        elif Y_test[i] == 1:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/test/1/' + filename[:-4] + '.png')
        else:
            print('error no label')

In [25]:
for i, sample in enumerate(sample_ids_train):
    prefix = sample[:-4]
    high_level_features = 0
    raw_feature = 'MFCCs20'
    result = [j for j in full_list if j.startswith(prefix)]
    for filename in result:
        # raw feature extraction:
        x = raw_feature_fromSample(path_save_audio_labels+filename, raw_feature ) # x.shape: (4, 20, 2584)
        x_norm = x

    #     #normalization here:
    #     if not normalization == 'NO':
    #         x_norm = featureMap_normalization_block_level(x, normalizationType = normalization) 
    #     else: x_norm = x

        if high_level_features:
            # high level feature extraction:
            if 'MFCCs' in raw_feature:
                X = compute_statistics_overMFCCs(x_norm, 'yes') # X.shape: (4 , 120)
            else: 
                X = compute_statistics_overSpectogram(x_norm)

            feature_map=X
        else:
            feature_map=x_norm

        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        p = librosa.display.specshow(feature_map, ax=ax, y_axis='log', x_axis='time')

        if Y_train[i] == 0:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/train/0/' + filename[:-4] + '.png')
        elif Y_train[i] == 1:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/train/1/' + filename[:-4] + '.png')
        else:
            print('error no label')

In [26]:
for i, sample in enumerate(sample_ids_val):
    prefix = sample[:-4]
    high_level_features = 0
    raw_feature = 'MFCCs20'
    result = [j for j in full_list if j.startswith(prefix)]
    for filename in result:
        # raw feature extraction:
        x = raw_feature_fromSample(path_save_audio_labels+filename, raw_feature ) # x.shape: (4, 20, 2584)
        x_norm = x

    #     #normalization here:
    #     if not normalization == 'NO':
    #         x_norm = featureMap_normalization_block_level(x, normalizationType = normalization) 
    #     else: x_norm = x

        if high_level_features:
            # high level feature extraction:
            if 'MFCCs' in raw_feature:
                X = compute_statistics_overMFCCs(x_norm, 'yes') # X.shape: (4 , 120)
            else: 
                X = compute_statistics_overSpectogram(x_norm)

            feature_map=X
        else:
            feature_map=x_norm

        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        p = librosa.display.specshow(feature_map, ax=ax, y_axis='log', x_axis='time')

        if Y_val[i] == 0:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/val/0/' + filename[:-4] + '.png')
        elif Y_val[i] == 1:
            fig.savefig('/Users/annayang/Documents/QueenBee/categorizedNewDataset/val/1/' + filename[:-4] + '.png')
        else:
            print('error no label')

In [29]:
from collections import Counter
print(Counter(Y_train))
print(Counter(Y_val))
print(Counter(Y_test))

Counter({0: 3441, 1: 439})
Counter({0: 843, 1: 127})
Counter({0: 470, 1: 69})


# image crop & save to 'croppedDataset'

In [38]:
path_images='/Users/annayang/Documents/QueenBee/imagesNewDataset'+os.sep
path_save_images='/Users/annayang/Documents/QueenBee/newCroppedDataset'+os.sep

In [40]:
for i, filename in enumerate(sample_ids_train):
    im = Image.open(path_images+'train/'+str(Y_train[i])+'/'+filename[:-4]+'.png')
    im_crop = im.crop((85, 70, 500, 400))
    im_crop.save((path_save_images+'train/'+str(Y_train[i])+'/'+filename[:-4]+'.png'), quality=100)

In [41]:
for i, filename in enumerate(sample_ids_test):
    im = Image.open(path_images+'test/'+str(Y_test[i])+'/'+filename[:-4]+'.png')
    im_crop = im.crop((85, 70, 500, 400))
    im_crop.save((path_save_images+'test/'+str(Y_test[i])+'/'+filename[:-4]+'.png'), quality=100)

In [42]:
for i, filename in enumerate(sample_ids_val):
    im = Image.open(path_images+'val/'+str(Y_val[i])+'/'+filename[:-4]+'.png')
    im_crop = im.crop((85, 70, 500, 400))
    im_crop.save((path_save_images+'val/'+str(Y_val[i])+'/'+filename[:-4]+'.png'), quality=100)

# create test set as hive

In [43]:
old_data = pd.read_csv('/Users/annayang/Documents/QueenBee/data_combined_training_1.csv')
old_data

,Unnamed: 0,device,date,hive temp,hive humidity,hive pressure,weather temp,weather humidity,weather pressure,wind speed,...,weatherID,cloud coverage,rain,lat,long,file name,queen presence,queen acceptance,frames,target
0,0,1,2022-06-08 14:52:28,36.42,30.29,1007.45,26.68,52,1013,8.75,...,711,75,0,37.29,-121.95,2022-06-08--14-52-28_1.raw,1,2,8,5
1,1,1,2022-06-08 15:51:41,33.56,33.98,1006.93,25.99,53,1012,10.29,...,803,75,0,37.29,-121.95,2022-06-08--15-51-41_1.raw,1,2,8,5
2,2,1,2022-06-08 17:21:53,29.01,42.73,1006.68,24.49,56,1012,8.75,...,803,75,0,37.29,-121.95,2022-06-08--17-21-53_1.raw,0,0,8,0
3,3,1,2022-06-08 18:20:59,30.51,36.74,1006.68,22.97,59,1012,8.23,...,803,75,0,37.29,-121.95,2022-06-08--18-20-59_1.raw,0,0,8,0
4,4,1,2022-06-08 19:20:04,30.32,35.55,1006.58,21.52,61,1012,7.20,...,803,75,0,37.29,-121.95,2022-06-08--19-20-04_1.raw,0,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,281,2,2022-06-13 12:48:18,47.41,9.88,1010.25,23.59,40,1016,7.72,...,800,0,0,37.29,-121.95,2022-06-13--12-48-18_2.raw,1,1,10,4
282,282,2,2022-06-13 13:47:23,41.90,12.83,1009.91,24.34,40,1016,6.17,...,800,0,0,37.29,-121.95,2022-06-13--13-47-23_2.raw,1,1,10,4
283,283,2,2022-06-13 14:46:29,33.34,18.27,1009.49,24.62,37,1016,7.72,...,800,0,0,37.29,-121.95,2022-06-13--14-46-29_2.raw,1,1,10,4
284,284,2,2022-06-13 15:45:41,30.43,21.06,1009.36,24.19,35,1016,8.75,...,800,0,0,37.29,-121.95,2022-06-13--15-45-41_2.raw,1,1,10,4


In [46]:
block_size=60
old_path_save_audio_labels='/Users/annayang/Documents/QueenBee/MyDataset'+os.sep  # path where to save audio segments and labels files.
old_path_raw_state_labels = '/Users/annayang/Documents/QueenBee/MyDataset/raw_state_labels.csv'
old_path_state_labels = '/Users/annayang/Documents/QueenBee/MyDataset/state_labels.csv'

In [65]:
path_images='/Users/annayang/Documents/QueenBee/imagesDataset'+os.sep
path_save_images='/Users/annayang/Documents/QueenBee/testHiveDataset'+os.sep

In [44]:
Counter(old_data['device'])

Counter({1: 97, 2: 189})

In [45]:
Counter(new_data['device'])

Counter({1: 462, 2: 513})

In [47]:
# old_sample_ids_test, old_sample_ids_train, old_sample_ids_val = get_samples_id_perSet(old_path_save_audio_labels+'split_random_0.json')

# labels2read = 'state_labels'
# labels_train = get_GT_labels_fromFiles(old_path_save_audio_labels, old_sample_ids_train, labels2read)
# old_Y_train= labels2binary('missing queen', labels_train)

# labels_val = get_GT_labels_fromFiles(old_path_save_audio_labels, old_sample_ids_val, labels2read)
# old_Y_val= labels2binary('missing queen', labels_val)

# labels_test = get_GT_labels_fromFiles(old_path_save_audio_labels, old_sample_ids_test, labels2read)
# old_Y_test= labels2binary('missing queen', labels_test)

In [80]:
# moving into test data
for i in os.listdir(path_images):
    fileprefix = i[:22]
    if fileprefix[-1:] == '1':
        presence = 0
        if int(old_data.loc[old_data['file name'] == fileprefix + '.raw']['queen presence']) == 0:
            presence = 1
        im = Image.open(path_images+i[:-4]+'.png')
        im.save((path_save_images+'test/'+str(presence)+'/'+i[:-4]+'.png'), quality=100)

In [82]:
# regenerating split for the new data w/out test data
split_dict = split_samples_ramdom(0,0.15,path_save_audio_labels, '/Users/annayang/Documents/QueenBee/'+'testhive_split_random_1')

In [108]:
Counter(Y_train)

Counter({0: 4045, 1: 536})

In [36]:
path_images = '/Users/annayang/Documents/QueenBee/imagesNewDataset'+os.sep
path_save_audio_labels = '/Users/annayang/Documents/QueenBee/newDataset/'

In [37]:
#regenerating sample ids
sample_ids_test, sample_ids_train, sample_ids_val = get_samples_id_perSet('/Users/annayang/Documents/QueenBee/testhive_split_random_1.json')

labels2read = 'state_labels'
labels_train = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_train, labels2read)
Y_train= labels2binary('missing queen', labels_train)

labels_val = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_val, labels2read)
Y_val= labels2binary('missing queen', labels_val)

labels_test = get_GT_labels_fromFiles(path_save_audio_labels, sample_ids_test, labels2read)
Y_test= labels2binary('missing queen', labels_test)

In [40]:
len(Y_train)

4581

In [42]:
train= 536 + 4045
val= 99 + 709
test= 149 + 432

In [45]:
val/(train + val + test)

0.1353433835845896

In [111]:
for i, filename in enumerate(sample_ids_train):
    im = Image.open(path_images+filename[:-4]+'.png')
    im.save((path_save_images+'train/'+str(Y_train[i])+'/'+filename[:-4]+'.png'), quality=100)

In [110]:
for i, filename in enumerate(sample_ids_val):
    im = Image.open(path_images+filename[:-4]+'.png')
    im.save((path_save_images+'val/'+str(Y_val[i])+'/'+filename[:-4]+'.png'), quality=100)

# queen accept/reject dataset

In [2]:
# queen status combined dataset
all_data_updated = pd.read_csv('/Users/annayang/Documents/QueenBee/all_data_zy.csv')
all_data_updated

,device,hive number,date,hive temp,hive humidity,hive pressure,weather temp,weather humidity,weather pressure,wind speed,...,rain,lat,long,file name,queen presence,queen acceptance,frames,target,time,queen status
0,1,5,2022-06-08 14:52:28,36.42,30.29,1007.45,26.68,52,1013,8.75,...,0,37.29,-121.95,2022-06-08--14-52-28_1.raw,1,2,8,0,0.583,0
1,1,5,2022-06-08 15:51:41,33.56,33.98,1006.93,25.99,53,1012,10.29,...,0,37.29,-121.95,2022-06-08--15-51-41_1.raw,1,2,8,0,0.625,0
2,1,5,2022-06-08 17:21:53,29.01,42.73,1006.68,24.49,56,1012,8.75,...,0,37.29,-121.95,2022-06-08--17-21-53_1.raw,0,0,8,1,0.708,1
3,1,5,2022-06-08 18:20:59,30.51,36.74,1006.68,22.97,59,1012,8.23,...,0,37.29,-121.95,2022-06-08--18-20-59_1.raw,0,0,8,1,0.750,1
4,1,5,2022-06-08 19:20:04,30.32,35.55,1006.58,21.52,61,1012,7.20,...,0,37.29,-121.95,2022-06-08--19-20-04_1.raw,0,0,8,1,0.792,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,2,1,2022-07-15 11:18:39,50.84,11.99,1010.21,23.58,55,1015,2.57,...,0,37.29,-121.95,2022-07-15--11-18-39_2.raw,1,2,10,5,0.458,3
1271,2,1,2022-07-15 12:25:14,49.58,11.60,1009.81,25.60,51,1015,4.12,...,0,37.29,-121.95,2022-07-15--12-25-14_2.raw,1,2,10,5,0.500,3
1272,2,1,2022-07-15 13:25:58,45.83,15.36,1009.80,26.49,49,1015,5.14,...,0,37.29,-121.95,2022-07-15--13-25-58_2.raw,1,2,10,5,0.542,3
1273,2,1,2022-07-15 14:24:58,35.82,23.48,1009.26,27.33,46,1014,4.63,...,0,37.29,-121.95,2022-07-15--14-24-58_2.raw,1,2,10,5,0.583,3


In [201]:
#get path for testHiveDataset train sample ids
path = "/Users/annayang/Documents/QueenBee/testHiveDataset_status"
new_path = "/Users/annayang/Documents/QueenBee/testHiveDataset_acc/"
train_dir_list_1 = os.listdir(path+'/train/3')
train_dir_list_0 =  os.listdir(path+'/train/2')
train_dir_list_0

['2022-06-29--23-53-49_2__segment0.png',
 '2022-06-30--09-46-45_2__segment3.png',
 '2022-06-29--12-16-28_2__segment0.png',
 '2022-06-28--14-10-34_2__segment1.png',
 '2022-06-29--16-11-49_1__segment4.png',
 '2022-06-30--00-53-00_2__segment2.png',
 '2022-06-29--02-22-49_1__segment5.png',
 '2022-06-28--23-25-29_2__segment2.png',
 '2022-06-28--05-16-54_1__segment0.png',
 '2022-06-29--06-19-34_2__segment0.png',
 '2022-06-28--00-21-03_1__segment1.png',
 '2022-06-29--06-19-33_1__segment1.png',
 '2022-06-29--14-13-31_2__segment4.png',
 '2022-06-29--01-23-48_2__segment5.png',
 '2022-06-28--04-17-44_1__segment5.png',
 '2022-06-29--17-59-12_2__segment0.png',
 '2022-06-28--20-28-09_2__segment4.png',
 '2022-06-30--02-51-21_2__segment2.png',
 '2022-06-30--02-51-21_2__segment3.png',
 '2022-06-28--20-28-09_2__segment5.png',
 '2022-06-29--17-59-12_2__segment1.png',
 '2022-06-29--01-23-48_2__segment4.png',
 '2022-06-28--04-17-44_1__segment4.png',
 '2022-06-29--19-57-13_2__segment4.png',
 '2022-06-29--14

In [202]:
for i in train_dir_list_0:
    im = Image.open(path+'/train/2/'+i)
    im.save((new_path+'train/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)
for i in train_dir_list_1:
    im = Image.open(path+'/train/3/'+i)
    im.save((new_path+'train/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)

In [203]:
val_dir_list_1 = os.listdir(path+'/val/3')
val_dir_list_0 =  os.listdir(path+'/val/2')
for i in val_dir_list_0:
    im = Image.open(path+'/val/2/'+i)
    im.save((new_path+'val/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)
for i in val_dir_list_1:
    im = Image.open(path+'/val/3/'+i)
    im.save((new_path+'val/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)

In [204]:
test_dir_list_1 = os.listdir(path+'/test/3')
test_dir_list_0 =  os.listdir(path+'/test/2')
for i in test_dir_list_0:
    im = Image.open(path+'/test/2/'+i)
    im.save((new_path+'test/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)
for i in test_dir_list_1:
    im = Image.open(path+'/test/3/'+i)
    im.save((new_path+'test/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)

In [205]:
already_recorded = train_dir_list_0 + train_dir_list_1 + val_dir_list_1 + val_dir_list_0 + test_dir_list_1 + test_dir_list_0
len(already_recorded)

4541

In [206]:
oldpath = "/Users/annayang/Documents/QueenBee/croppedDataset"
old_dir_train = os.listdir(oldpath+'/train/0')
old_dir_val = os.listdir(oldpath+'/val/0')
old_dir_test = os.listdir(oldpath+'/test/0')
# put all the test data into train, since it's missing the ones from the hive that's being used for new test
for i in old_dir_train:
    if i in already_recorded:
        continue
    else:
        im = Image.open(oldpath+'/train/0/'+i)
        im.save((new_path+'train/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)
for i in old_dir_val:
    if i in already_recorded:
        continue
    else:
        im = Image.open(oldpath+'/val/0/'+i)
        im.save((new_path+'val/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)
for i in old_dir_test:
    if i in already_recorded:
        continue
    else:
        im = Image.open(oldpath+'/test/0/'+i)
        im.save((new_path+'train/'+str(list(all_data_updated.loc[all_data_updated['file name'] == i[:-14]+'.raw']['queen acceptance'])[0])+"/"+i), quality=100)

# balancing the acceptance dataset

In [209]:
from random import sample
del_path = "/Users/annayang/Documents/QueenBee/testHiveDatasetBalanced_acc/val/1/"
files = os.listdir(del_path)
for file in sample(files,200):
    os.remove(del_path + file)